### Split each AMNet attention map triplet into three separate 224x224 images
Note: Training frames amount to approximately 700MB, downloadable from https://s3.console.aws.amazon.com/s3/object/c684i-2021?region=eu-west-1&prefix=train_frames.zip

In [4]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import os
from tensorflow.keras.preprocessing import image as imgproc
FRAME_INDEXES = [0, 24, 48, 72, 96, 120, 144]
SIZE = (224, 224)
# load data
base_dir = "/workspace/C684_Assignment/"
processing_dir = os.path.join(base_dir, "processing")
dev_videos = pd.read_csv(os.path.join(processing_dir, "ground_truth_template.csv"))# "amnet_devset_preds.csv"))
frames_dir = os.path.join(processing_dir, "att_maps")
train_dir = os.path.join(base_dir, "training")

# Training on the 2nd AMNet heatmap, so crop and save
train_frame_out_dir = os.path.join(train_dir, "train_frames-0")
if not os.path.exists(train_frame_out_dir):
    os.makedirs(train_frame_out_dir)
videos = dev_videos["video"]
with tqdm(total=len(videos)) as pbar:
    for f in videos:
        for i in FRAME_INDEXES:
            filename = "video%s_%d_att.jpg" % (f, i)
            img = imgproc.load_img(os.path.join(frames_dir, filename))
            width = (img.size[0] - 120) / 3
            heigth = img.size[1] - 90
            frame_crop_rect = (60+width, 20, 60+2*width, heigth+20) # crop rect for middle image
            img = img.crop(frame_crop_rect)
            img = img.resize(SIZE)
            img.save(os.path.join(train_frame_out_dir, filename))
        pbar.update(1)

100%|██████████| 2000/2000 [10:02<00:00,  3.32it/s]
